# Import

In [1]:
# evaluate knn on the sonar dataset with minmax scaler transform
from numpy import mean
from numpy import std
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from matplotlib import pyplot

# Code

In [24]:
n = 168605
s = 80000 #desired sample size
skip = sorted(random.sample(range(1,n),n-s))
main_dataset = pd.read_csv("../processed_files/main_dataset.csv", index_col=[0], skiprows=skip)
main_dataset = pd.read_csv("../processed_files/main_dataset.csv", index_col=[0])

NameError: name 'random' is not defined

In [25]:
#main_dataset = pd.read_csv("../processed_files/main_dataset.csv", index_col=[0])
#Drop Columns
main_dataset = main_dataset.drop(['codice_stazione','orario_arrivo','orario_partenza','orario',
                                  'trip_id','arrival_time','departure_time','codice'
                                 ], axis=1)


#main_dataset = main_dataset.loc[(main_dataset.ritardo_arrivo >= 1)]
#main_dataset = main_dataset.loc[(main_dataset.temperature <= 0)]
main_dataset = main_dataset.loc[(main_dataset.rain >= 0.5) ]

#Reset Index
main_dataset = main_dataset.reset_index(drop=True)

#Round Columns
main_dataset = main_dataset.round({'temperature':0,'speed':0,'wind_speed':0,'temperature_linea':0,
                                  'app_temp_linea':0,'wind_speed_linea':0}) 

KeyError: "['codice_stazione', 'orario_arrivo', 'orario_partenza', 'orario', 'trip_id', 'arrival_time', 'departure_time', 'codice'] not found in axis"

In [17]:
print(main_dataset.head())

   stop_sequence  shape_dist_traveled  time_diff  speed  dist_diff  \
0              1              0.00000       0.00    0.0       0.00   
1              2              8.01853       0.10   80.0       8.02   
2              1              0.00000       0.00    0.0       0.00   
3              2              8.01853       0.10   80.0       8.02   
4              3             14.83468       0.13   51.0       6.82   

   direction_id  data_giorno   stop_lat   stop_lon  stop_code  ...  rain  \
0             0     20201217  43.837397  10.506153       6404  ...   0.5   
1             0     20201217  43.821737  10.415844       6402  ...   0.5   
2             0     20201223  43.837397  10.506153       6404  ...   0.8   
3             0     20201223  43.821737  10.415844       6402  ...   0.8   
4             0     20201223  43.764855  10.436916       6400  ...   0.5   

   snow_fall  snow_depth  wind_speed  temperature_linea  app_temp_linea  \
0        0.0         0.0         8.0           

In [18]:
x = main_dataset.corr()

In [19]:
with pd.option_context('display.max_colwidth', None,
                       'display.max_columns', None,
                       'display.max_rows', None):
    display(x)

,stop_sequence,shape_dist_traveled,time_diff,speed,dist_diff,direction_id,data_giorno,stop_lat,stop_lon,stop_code,ritardo_arrivo,ritardo_partenza,temperature,app_temp,rain,snow_fall,snow_depth,wind_speed,temperature_linea,app_temp_linea,rain_linea,snow_fall_linea,snow_depth_linea,wind_speed_linea
stop_sequence,1.000000,0.969354,0.069207,0.223812,0.013274,0.339506,0.040026,0.472629,-0.093304,-0.431076,0.199392,0.102354,-0.019899,-0.007826,0.003230,0.019299,0.037629,-0.128917,-0.024889,-0.009229,0.005013,0.035125,0.053252,-0.154262
shape_dist_traveled,0.969354,1.000000,0.188130,0.266813,0.134803,0.358321,0.045063,0.451778,-0.064282,-0.415707,0.198844,0.092564,-0.017878,-0.005020,-0.000892,0.020122,0.037372,-0.130171,-0.022409,-0.005947,-0.000382,0.035186,0.054334,-0.158164
time_diff,0.069207,0.188130,1.000000,0.412732,0.870687,0.102026,0.005494,-0.000603,-0.015482,-0.017559,0.050172,-0.018214,0.007718,0.010140,-0.007878,-0.001194,0.009119,-0.015107,0.013432,0.013669,-0.032054,-0.000640,0.014298,-0.015028
speed,0.223812,0.266813,0.412732,1.000000,0.666064,0.111145,-0.006426,0.199831,-0.046917,-0.347567,0.201452,0.122863,0.004230,0.015529,0.006195,0.014984,0.029997,-0.090288,0.004025,0.013319,-0.013519,0.015098,0.024413,-0.088254
dist_diff,0.013274,0.134803,0.870687,0.666064,1.000000,0.096393,-0.007296,0.013722,0.014287,-0.108451,0.069516,0.018190,0.020206,0.021595,0.001834,0.004043,0.012771,-0.005718,0.024650,0.022857,-0.026356,0.002620,0.010744,-0.001082
direction_id,0.339506,0.358321,0.102026,0.111145,0.096393,1.000000,0.032797,0.410601,-0.184472,-0.409768,0.027692,0.000357,-0.023841,-0.016492,0.013412,0.013732,0.040272,-0.082659,-0.024475,-0.016861,0.003137,0.017658,0.044539,-0.092162
data_giorno,0.040026,0.045063,0.005494,-0.006426,-0.007296,0.032797,1.000000,0.010244,0.048400,-0.025912,-0.061677,-0.053390,0.200787,0.200222,0.038098,0.041193,0.033589,-0.073655,0.203163,0.201409,0.027774,0.043774,0.034273,-0.071929
stop_lat,0.472629,0.451778,-0.000603,0.199831,0.013722,0.410601,0.010244,1.000000,-0.553719,-0.890650,0.145362,0.108240,-0.156372,-0.135576,-0.006723,0.010220,0.077060,-0.172571,-0.152855,-0.132217,-0.018361,0.014841,0.078901,-0.179491
stop_lon,-0.093304,-0.064282,-0.015482,-0.046917,0.014287,-0.184472,0.048400,-0.553719,1.000000,0.443592,-0.033163,-0.005468,0.132356,0.138353,0.030251,0.033107,-0.048090,-0.007866,0.130900,0.137552,0.048414,0.033353,-0.049508,-0.000209
stop_code,-0.431076,-0.415707,-0.017559,-0.347567,-0.108451,-0.409768,-0.025912,-0.890650,0.443592,1.000000,-0.149716,-0.158422,0.111665,0.086726,-0.007675,-0.023564,-0.084287,0.202564,0.109687,0.087489,0.006895,-0.024255,-0.081692,0.196534


In [20]:
main_dataset.describe()


,stop_sequence,shape_dist_traveled,time_diff,speed,dist_diff,direction_id,data_giorno,stop_lat,stop_lon,stop_code,...,rain,snow_fall,snow_depth,wind_speed,temperature_linea,app_temp_linea,rain_linea,snow_fall_linea,snow_depth_linea,wind_speed_linea
count,25429.000000,25429.000000,25429.000000,25429.000000,25429.000000,25429.000000,2.542900e+04,25429.000000,25429.000000,25429.000000,...,25429.000000,25429.000000,25429.000000,25429.000000,25429.000000,25429.000000,25429.000000,25429.000000,25429.000000,25429.000000
mean,6.959652,29.076660,0.085763,49.090959,4.594387,0.308939,2.021773e+07,43.970386,10.394962,6344.372056,...,1.772347,0.006312,0.000351,9.267372,14.116088,13.502301,1.687906,0.006759,0.000351,9.289473
std,5.231402,23.055693,0.046264,20.485323,2.751834,0.462065,8.431374e+03,0.170909,0.133965,81.583769,...,1.870386,0.084430,0.002623,5.354718,5.250869,6.723994,1.781459,0.082795,0.002543,5.226136
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.020122e+07,43.708231,10.061017,6222.000000,...,0.500000,0.000000,0.000000,0.000000,2.000000,-3.000000,0.250000,0.000000,0.000000,0.000000
25%,3.000000,11.307860,0.070000,40.000000,2.620000,0.000000,2.021052e+07,43.821737,10.350640,6304.000000,...,0.700000,0.000000,0.000000,5.000000,10.000000,9.000000,0.600000,0.000000,0.000000,5.000000
50%,5.000000,23.307570,0.080000,52.000000,4.130000,0.000000,2.022033e+07,43.977306,10.415844,6322.000000,...,1.100000,0.000000,0.000000,8.000000,13.000000,12.000000,1.100000,0.000000,0.000000,8.000000
75%,10.000000,44.937420,0.120000,62.000000,6.820000,1.000000,2.022122e+07,44.148485,10.506153,6402.000000,...,2.100000,0.000000,0.000000,12.000000,17.000000,18.000000,2.050000,0.000000,0.000000,12.000000
max,23.000000,89.915800,0.430000,96.000000,20.910000,1.000000,2.023103e+07,44.202877,10.551756,6501.000000,...,26.200000,3.220000,0.060000,36.000000,34.000000,38.000000,26.200000,2.940000,0.060000,36.000000


In [21]:


#main_dataset = main_dataset.loc[(main_dataset.ritardo_arrivo >= 1)]
#main_dataset = main_dataset.loc[(main_dataset.temperature <= 0)]
#main_dataset = main_dataset.loc[(main_dataset.rain >= 0.5) ]

#Reset Index
main_dataset.reset_index(drop=True)
#main_dataset = main_dataset.loc[main_dataset.ritardo_totale < 1000]


,stop_sequence,shape_dist_traveled,time_diff,speed,dist_diff,direction_id,data_giorno,stop_lat,stop_lon,stop_code,...,rain,snow_fall,snow_depth,wind_speed,temperature_linea,app_temp_linea,rain_linea,snow_fall_linea,snow_depth_linea,wind_speed_linea
0,1,0.00000,0.00,0.0,0.00,0,20201217,43.837397,10.506153,6404,...,0.5,0.0,0.0,8.0,11.0,10.0,0.50,0.0,0.0,8.0
1,2,8.01853,0.10,80.0,8.02,0,20201217,43.821737,10.415844,6402,...,0.5,0.0,0.0,8.0,11.0,10.0,0.50,0.0,0.0,8.0
2,1,0.00000,0.00,0.0,0.00,0,20201223,43.837397,10.506153,6404,...,0.8,0.0,0.0,14.0,13.0,12.0,0.80,0.0,0.0,14.0
3,2,8.01853,0.10,80.0,8.02,0,20201223,43.821737,10.415844,6402,...,0.8,0.0,0.0,14.0,13.0,12.0,0.80,0.0,0.0,14.0
4,3,14.83468,0.13,51.0,6.82,0,20201223,43.764855,10.436916,6400,...,0.5,0.0,0.0,10.0,13.0,12.0,0.65,0.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25424,1,0.00000,0.00,0.0,0.00,0,20231030,43.837397,10.506153,6404,...,5.3,0.0,0.0,14.0,19.0,19.0,5.30,0.0,0.0,14.0
25425,2,8.01853,0.10,80.0,8.02,0,20231030,43.821737,10.415844,6402,...,5.3,0.0,0.0,14.0,19.0,19.0,5.30,0.0,0.0,14.0
25426,3,14.83468,0.13,51.0,6.82,0,20231030,43.764855,10.436916,6400,...,5.3,0.0,0.0,12.0,19.0,20.0,5.30,0.0,0.0,13.0
25427,4,20.90887,0.15,40.0,6.07,0,20231030,43.723446,10.387754,6501,...,1.7,0.0,0.0,23.0,19.0,19.0,3.50,0.0,0.0,18.0


In [22]:
nan_df = main_dataset.isna().any(axis=0)

print(nan_df)

stop_sequence          False
shape_dist_traveled    False
time_diff              False
speed                  False
dist_diff              False
direction_id           False
data_giorno            False
stop_lat               False
stop_lon               False
stop_code              False
ritardo_arrivo         False
ritardo_partenza       False
temperature            False
app_temp               False
rain                   False
snow_fall              False
snow_depth             False
wind_speed             False
temperature_linea      False
app_temp_linea         False
rain_linea             False
snow_fall_linea        False
snow_depth_linea       False
wind_speed_linea       False
dtype: bool


In [23]:
data_input = main_dataset.drop(['ritardo_partenza'], axis=1)
# separate into input and output columns
X = data_input.values
y = main_dataset.ritardo_partenza.values

# ensure inputs are floats and output is an integer label
X = X.astype('float32')
y = LabelEncoder().fit_transform(y.astype('str'))

# define the pipeline
trans = MinMaxScaler()
model = KNeighborsRegressor(n_neighbors=5)
pipeline = Pipeline(steps=[('t', trans), ('m', model)])

# evaluate the pipeline
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
# report pipeline performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/Users/gianlucarea/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/Users/gianlucarea/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/Users/gianlucarea/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/Users/gianlucarea/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/Users/gianlucarea/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, whi

Accuracy: 0.191 (0.008)
